In [2]:
import numpy as np
import pandas as pd
import math
from arbiter_puf import ArbiterPuf
from xor_puf import XorPuf
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score


In [3]:
#  drops responses in Y  adds responses in X
def modify_data_to_train(data):
    training_data = []
    test_data = []
    for i in data:
        test_data.append(i.pop())
        training_data.append(i)
    return np.asarray(training_data), np.asarray(test_data)

In [23]:
list_of_number_of_bits = [32, 64]
data_size = 100000
print('================== Arbiter PUF ========================')
for bit in list_of_number_of_bits:
    print('Number of Stages: ', bit)
    dataset = ArbiterPuf(bit).calculate_responses(data_size)
    X,Y = modify_data_to_train(dataset)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    LGR_Classifier = LogisticRegression()
    LGR_Classifier.fit(X_train, Y_train)
    scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
    y_pred = LGR_Classifier.predict(X_test)
    print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
    print("Accuracy : ", accuracy_score(Y_test, y_pred))
    print(classification_report(Y_test, y_pred))
    print('=======================================================')


================== Arbiter PUF ========================
Number of Stages:  32
Cross Validation Mean Score:  50.1%
Accuracy :  0.50215
              precision    recall  f1-score   support

           0       0.50      0.48      0.49     10048
           1       0.50      0.52      0.51      9952

    accuracy                           0.50     20000
   macro avg       0.50      0.50      0.50     20000
weighted avg       0.50      0.50      0.50     20000

Number of Stages:  64
Cross Validation Mean Score:  56.39999999999999%
Accuracy :  0.56905
              precision    recall  f1-score   support

           0       0.58      0.57      0.57     10113
           1       0.56      0.57      0.57      9887

    accuracy                           0.57     20000
   macro avg       0.57      0.57      0.57     20000
weighted avg       0.57      0.57      0.57     20000



In [4]:
def modify_data(data): # adding phi to the dataset when training
    training_data = []
    test_data = []
    for i in data:
        test_data.append(i.pop())
        phi = [ArbiterPuf.calculate_phi(i[j:]) for j in range(len(i))]
        training_data.append(phi)   
    return np.asarray(training_data), np.asarray(test_data)


In [5]:
list_of_number_of_bits = [32, 64]
data_size = 100000
print('================== Arbiter PUF with Phi ========================')
for bit in list_of_number_of_bits:
    print('Number of Stages: ', bit)
    dataset = ArbiterPuf(bit).calculate_responses(data_size)
    X,Y = modify_data(dataset)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    LGR_Classifier = LogisticRegression()
    LGR_Classifier.fit(X_train, Y_train)
    scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
    y_pred = LGR_Classifier.predict(X_test)
    print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
    print("Accuracy : ", accuracy_score(Y_test, y_pred))
    print(classification_report(Y_test, y_pred))
    print('=======================================================')


================== Arbiter PUF with Phi ========================
Number of Stages:  32
Cross Validation Mean Score:  99.9%
Accuracy :  0.9985
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10037
           1       1.00      1.00      1.00      9963

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

Number of Stages:  64
Cross Validation Mean Score:  99.9%
Accuracy :  0.99775
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10061
           1       1.00      1.00      1.00      9939

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000



In [9]:
list_of_number_of_bits = [5, 10, 15, 20]
data_size = 100000
number_of_pufs = [3, 4, 5]
for j in number_of_pufs:
    print(f"================== Xor PUF {j} Pufs ========================")
    for bit in list_of_number_of_bits:
        print('Number of Stages: ', bit)
        # dataset = ArbiterPuf(bit).calculate_responses(data_size)
        dataset = XorPuf(bit, j).calculate_responses(data_size)
        X,Y = modify_data(dataset)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        LGR_Classifier = LogisticRegression()
        LGR_Classifier.fit(X_train, Y_train)
        scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
        y_pred = LGR_Classifier.predict(X_test)
        print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
        print("Accuracy : ", accuracy_score(Y_test, y_pred))
        print(classification_report(Y_test, y_pred))
        print('=======================================================')


================== Xor PUF 3 Pufs ========================
Number of Stages:  5
Cross Validation Mean Score:  75.0%
Accuracy :  0.7473
              precision    recall  f1-score   support

           0       0.75      0.75      0.75     10072
           1       0.75      0.75      0.75      9928

    accuracy                           0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000

Number of Stages:  10
Cross Validation Mean Score:  55.900000000000006%
Accuracy :  0.5598
              precision    recall  f1-score   support

           0       0.56      0.55      0.56     10011
           1       0.56      0.57      0.56      9989

    accuracy                           0.56     20000
   macro avg       0.56      0.56      0.56     20000
weighted avg       0.56      0.56      0.56     20000

Number of Stages:  15
Cross Validation Mean Score:  56.699999999999996%
Accuracy :  0.57285
              precision    r

/home/v/PycharmProjects/puf/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Cross Validation Mean Score:  51.7%
Accuracy :  0.52205
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      9545
           1       0.52      1.00      0.69     10455

    accuracy                           0.52     20000
   macro avg       0.26      0.50      0.34     20000
weighted avg       0.27      0.52      0.36     20000

Number of Stages:  15
Cross Validation Mean Score:  50.0%
Accuracy :  0.5045
              precision    recall  f1-score   support

           0       0.51      0.93      0.66     10269
           1       0.43      0.06      0.10      9731

    accuracy                           0.50     20000
   macro avg       0.47      0.49      0.38     20000
weighted avg       0.47      0.50      0.39     20000

Number of Stages:  20
Cross Validation Mean Score:  52.900000000000006%
Accuracy :  0.52765
              precision    recall  f1-score   support

           0       0.53      1.00      0.69     10553
           1

/home/v/PycharmProjects/puf/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Cross Validation Mean Score:  62.9%
Accuracy :  0.62285
              precision    recall  f1-score   support

           0       0.63      0.62      0.62     10119
           1       0.62      0.63      0.62      9881

    accuracy                           0.62     20000
   macro avg       0.62      0.62      0.62     20000
weighted avg       0.62      0.62      0.62     20000

Number of Stages:  10
Cross Validation Mean Score:  58.9%
Accuracy :  0.59115
              precision    recall  f1-score   support

           0       0.59      0.60      0.60     10096
           1       0.59      0.58      0.58      9904

    accuracy                           0.59     20000
   macro avg       0.59      0.59      0.59     20000
weighted avg       0.59      0.59      0.59     20000

Number of Stages:  15
Cross Validation Mean Score:  54.7%
Accuracy :  0.54515
              precision    recall  f1-score   support

           0       0.55      0.54      0.54      9992
           1       0.55  